In [ ]:
import pandas as pd
x=pd.read_csv('winemag_p1.csv')

In [ ]:
# x[['variety','winery']].head(60).sort_values('winery') 

In [ ]:
# Total Wine Revenue
# You have a dataset of wines. Find the total revenue made by each winery and variety that has at least 90 points.
# Each wine in the winery, variety pair should be at least 90 points in order for the wine to be considered in the calculation.
# Output the winery and variety along with the corresponding total revenue. Order records by the winery in ascending order and total revenue in descending order.

main = x.groupby(['variety','winery']).agg({'price':'sum','points':'min'}).reset_index()
final = main[main['points']>=90]
print(main)
final.drop(columns=['points'],axis=1).sort_values(by=['winery','price'],ascending=['True','False'])






import pandas as pd
import numpy as np
min_points = winemag_p1.groupby(['winery','variety']).agg({'price':'sum','points':'min'}).reset_index()
result = min_points[min_points['points'] >= 90].sort_values(['winery','price'], ascending = [True,False])[['winery','variety','price']]


SELECT winery, variety, sum(price) AS total_revenue
FROM winemag_p1
GROUP BY winery,  variety
HAVING min(points) >= 90
ORDER BY winery, total_revenue DESC

select winery, variety, sum(price) as tot_revenue
from winemag_p1
group by winery, variety
having min(points) >= 90
order by winery, tot_revenue desc

In [ ]:
# Highest Priced Wine In The US
# Find the highest price of each variety produced in English speaking regions, but not in Spanish speaking regions,
# and considering varieties that have earned a minimum of 90 points for every country. 
# Use the result to find the highest price of each variety in the US.
# Output the variety and the corresponding highest price.
# Lets assume the US is the only English speaking region in the dataset, and Spain, Argentina, Portugal are the only Spanish speaking regions in the dataset.
# Lets also assume that the same variety might be listed under several countries so youll need to remove varieties that show up in both the US and in Spanish speaking countries.

# Import your libraries
import pandas as pd
winemag_p1=pd.read_csv('winemag_p1.csv')
# Start writing code
winemag_p1.head()
df = winemag_p1
non_spanish_varities = df[~df.country.isin(['Spain', 'Argentina', 'Portugal'])].variety.unique()
df_=df[(df.variety.isin(non_spanish_varities))].groupby('variety').filter(lambda g: g['points'].min() >= 90)

df_[df_.country=='US'].groupby('variety').price.max().reset_index(name='price')



WITH A AS (select   variety 
from winemag_p1 
group by 1,country
having sum(case when country='US' then 1 else 0 end)>0
and  sum(case when country  in  ('Spain','Argentina','Portugal'  ) then 1 else 0 end)=0
   )
, B AS(  select variety  from winemag_p1
  where variety in (select * from a) 
  group by 1 
  having min(points)>=90 
  )
  select variety,price from winemag_p1 where variety in (select * from b)
    
    
    
    
    
    
SELECT variety,  max(price)
FROM winemag_p1
WHERE variety in
    (SELECT variety
     FROM winemag_p1
     WHERE variety not in
         (SELECT variety
          FROM winemag_p1
          WHERE country in ('Spain', 'Argentina', 'Portugal'))
     GROUP BY variety
     HAVING min(points) >= 90)
  AND country = 'US'
GROUP BY variety

In [ ]:
import pandas as pd
import numpy as np

spanish_region = ['Spain', 'Argentina', 'Portugal']
spanish_varieties = winemag_p1[winemag_p1['country'].isin(spanish_region)]['variety']
min_variety = winemag_p1.groupby(['variety'])['points'].min().to_frame('min_points_per_variety').reset_index()
variety_90 = min_variety[min_variety['min_points_per_variety'] >= 90]['variety']
result = winemag_p1[(winemag_p1['country'] == 'US') & (~winemag_p1['variety'].isin(spanish_varieties)) & (
    winemag_p1['variety'].isin(variety_90))].groupby('variety')['price'].max().reset_index()
result

In [ ]:
# Points Rating Of Wines Over Time
# Find the average points difference between each and previous years starting from the year 2000. 
# Output the year, average points, previous average points, and the difference between them.
# If youre unable to calculate the average points rating for a specific year, use an 87 average points rating for that year (which is the average of all wines starting from 2000).

import pandas as pd
winemag_p2=pd.read_csv('winemag_p2.csv')
# Start writing code
winemag_p2.head()
df = winemag_p2
df 
# Import your libraries
 
winemag_p2['year'] = (winemag_p2['title'].str.extract('(\d{4,4})')).fillna(0).astype(int)

winemag_p2 = winemag_p2[(winemag_p2['year'] >= 2000)& (winemag_p2['points'].notnull() )]
result = winemag_p2.groupby(['year'])['points'].mean().to_frame('avg_points').reset_index()
result['prev_avg'] = (result['avg_points'].shift(1)).fillna(87)
result['difference'] = result['avg_points'] - result['prev_avg']
print(result)





with a as (select substring(title, '\d{4}')::integer AS year,avg(points) avgpts
from winemag_p2
where points is not null
and title is not null
group by 1
order by 1 )
,b as ( select year,avgpts,COALESCE(lag(avgpts)over(),87) prev_avg_points
from a
 where year >1999 and year is not null
)
select year,avgpts as  avg_points, prev_avg_points,avgpts-prev_avg_points difference
from b

In [ ]:
# Price Of Wines In Each Country
# Find the minimum, average, and maximum price of all wines per country.
# Assume all wines listed across both datasets are unique.
# Output the country name along with the corresponding minimum, maximum, and average prices.

import pandas as pd
import numpy as np
winemag_p1=pd.read_csv('winemag_p1.csv')
winemag_p2=pd.read_csv('winemag_p2.csv')
# # Import your libraries
# import pandas as pd
# df1=winemag_p1[['country','price']]
# df2=winemag_p2[['country','price']]
# df=pd.concat([df1, df2], ignore_index=True).sort_values(by='country') 
# df.groupby('country').agg({'price': [ 'min','mean', 'max']}).reset_index()






p2 = winemag_p2[(winemag_p2['country'].notnull())][['country','price']]
p1 = winemag_p1[winemag_p1['country'].notnull()][['country','price']]
p2['price'] = p2['price'].replace('', np.nan)
p1['price'] = p1['price'].replace('', np.nan)
merged_df = pd.concat([p1,p2], axis = 0)
merged_df['price'] = merged_df['price'].astype(float)
result=merged_df.groupby(["country"])['price'].agg( MIN = 'min', AVG = 'mean', MAX = 'max').reset_index()
result


with a as(select country, price from winemag_p1
union all
select country, price from winemag_p2)
select country, min(price),avg(price),max(price) from a group by 1

In [ ]:
# Most Expensive And Cheapest Wine
# Find the cheapest and the most expensive variety in each region. 
# Output the region along with the corresponding most expensive and the cheapest variety.



# Import your libraries
import pandas as pd
df1 = winemag_p1[['region_1', 'price', 'variety']].rename(columns={'region_1':'region'})
print(df1)
df2 = winemag_p1[['region_2', 'price', 'variety']].rename(columns={'region_2':'region'})
df = pd.concat([df1, df2], ignore_index=True) 
 

cheap = df.groupby(['region']).apply(lambda x: x.nsmallest(1, 'price')).reset_index(drop=True)[['region','variety']]
cheap.columns=['region','cheap'] 
print(cheap,'cheap')
expensive = df.groupby(['region']).apply(lambda x: x.nlargest(1, 'price')).reset_index(drop=True)[['region','variety']]
expensive.columns=['region','expensive'] 
result = pd.merge(cheap, expensive, on = 'region') 
result.columns=['region','var_expensive','var_cheap']
result





# import pandas as pd
# import numpy as np
# df1 = winemag_p1[['region_1', 'price', 'variety']].rename(columns={'region_1':'region'})
# df2 = winemag_p1[['region_2', 'price', 'variety']].rename(columns={'region_2':'region'})
# df = pd.concat([df1, df2], ignore_index=True)
# expensive = df[~df['region'].isna()].groupby(['region']).apply(lambda x: x.nlargest(1, 'price')).reset_index(drop=True).rename(columns={'variety':'var_expensive'})
# cheap = df[~df['region'].isna()].groupby(['region']).apply(lambda x: x.nsmallest(1, 'price')).reset_index(drop=True).rename(columns={'variety':'var_cheap'})
# result = pd.merge(expensive,cheap, on = 'region')[['region','var_expensive','var_cheap']]                                     



with a as (select region_1,variety,price ,
rank()over(partition by region_1 order by price) high,
rank()over(partition by region_1 order by price desc) low
from winemag_p1
where price is not null
order by 1 
 ),
 b as (select region_1,variety from a where high =1),
c as (select region_1 ,variety  from a where low =1)
select  b.region_1,c.variety  most_expensive_variety,
 b.variety cheapest_variety
from b join c
using (region_1 )

In [ ]:
# Points Rating Of Wines Over Time
# Find the average points difference between each and previous years starting from the year 2000. 
# Output the year, average points, previous average points, and the difference between them.
# If youre unable to calculate the average points rating for a specific year, 
# use an 87 average points rating for that year (which is the average of all wines starting from 2000).

# Import your libraries
import pandas as pd

p1 = winemag_p1[(winemag_p1['variety'].notnull()) & (winemag_p1['price'].notnull())][['variety','price']]
p2 = winemag_p2[(winemag_p2['variety'].notnull()) & (winemag_p2['price'].notnull())][['variety','price']]    
df = pd.concat([p1,p2])
df=df.groupby('variety')['price'].median().to_frame('median_price').reset_index()
df.sort_values('median_price',ascending=False)







with a as (select substring(title, '\d{4}')::integer AS year,avg(points) avgpts
from winemag_p2
where points is not null
and title is not null
group by 1
order by 1 )
,b as ( select year,avgpts,COALESCE(lag(avgpts)over(),87) prev_avg_points
from a
 where year >1999 and year is not null
)
select year,avgpts as  avg_points, prev_avg_points,avgpts-prev_avg_points difference
from b

In [ ]:
# Top 3 Wineries In The World
# Find the top 3 wineries in each country based on the average points earned.
# Output the country along with the best, second best, and third best wineries.

# Import your libraries
import pandas as pd
# Start writing code
print(winemag_p1.info())

# winemag_p1.head()
df = winemag_p1.groupby(['country', 'winery'])['points'].mean().to_frame('avg_points').reset_index()
# print(df,'df')
top_3 = df.sort_values('avg_points', ascending = False).groupby('country').apply(lambda x: x.nlargest(3,'avg_points')).reset_index(drop=True)
# print(top_3,'top_3')
top_3['rank'] = top_3.groupby('country').cumcount()+1
# print(top_3,'top_3')
result = top_3.pivot(index='country', columns = 'rank', values = 'winery').reset_index()
result



with a as (select country,winery,  avg(points) avgpts
from winemag_p1
 group by 1,2
order by 1,3 desc)
,b as(select country,concat(winery,' (',avgpts::int,')') winery  ,
row_number() over(partition by country  order by avgpts desc) rn
from a),
 f as (select country,    winery   as top_winery from b  where rn=1),
 s as (select country,    winery   as second_winery from b  where rn=2),
 t as (select country,    winery   as third_winery from b  where rn=3)

select distinct  b.country  ,  top_winery , 
coalesce(second_winery,'No second winery') second_winery,  
coalesce(third_winery,'No third winery')third_winery
from  b left join f using (country)
   left join s using (country)
left join t using (country)
order by 1

In [ ]:
winemag_p1=pd.read_csv('winemag_p1.csv')
winemag_p2=pd.read_csv('winemag_p2.csv')

p1 = winemag_p1[(winemag_p1['variety'].notnull()) & (winemag_p1['price'].notnull())] 
p2 = winemag_p2[(winemag_p2['variety'].notnull()) & (winemag_p2['price'].notnull())]     
merged = pd.concat([p1,p2])
print(merged.info())
# Find the top 3 wineries in each country based on the average points earned. Output the country along with the best, second best, and third best wineries.

points = merged.groupby(['country','winery'])['points'].mean().reset_index()

points['rank'] = points.groupby('country')['points'].rank(method="first",ascending=False)
output=points[points['rank']<=3]

results = output.pivot_table(columns='rank',index='country', values='winery',aggfunc='first').reset_index() 
results

In [ ]:
# Find Favourite Wine Variety
# Find each taster's favorite wine variety. 
# Consider that favorite variety means the variety that has been tasted by most of the time.
# Output the taster's name along with the wine variety.
# Import your libraries
import pandas as pd

# Start writing code

t = winemag_p2[winemag_p2.taster_name.notnull()].groupby(['taster_name', 'variety']).size().to_frame('count').reset_index()
t['rnk'] = t.groupby(['taster_name'])['count'].rank(method = 'dense' , ascending = False)
t.loc[t.rnk == 1 , ['taster_name' , 'variety']]



with a as (select taster_name,variety,count(*),
rank()over(partition by taster_name order by count(*) desc) rn
from winemag_p2
where taster_name is not null
group by 1,2
order by 3 desc)
select taster_name,variety from a where rn =1

In [ ]:
import pandas as pd

# Start writing code
provinces_1 = winemag_p1.province.value_counts()
provinces_2 = winemag_p2.province.value_counts()

merged =pd.merge(left=provinces_1,right=provinces_2,left_index=True,right_index=True,suffixes=('_1','_2'))
merged = merged.reset_index()
merged[merged.province_1 > merged.province_2].drop('province_2',axis=1)

In [ ]:
winemag_p1.head()
winemag_p1=pd.read_csv('winemag_p1.csv')
winemag_p2=pd.read_csv('winemag_p2.csv')
# Find all provinces which produced more wines in 'winemag_p1' than they did in 'winemag_p2'.
# Output the province and the corresponding wine count.

import pandas as pd
import numpy as np

p1 = winemag_p1[winemag_p1['province'].notnull()]
p2 = winemag_p2[winemag_p2['province'].notnull()]
n_p1 = p1.groupby(['province']).size().to_frame('count_p1').reset_index()
n_p2 = p2.groupby(['province']).size().to_frame('count_p2').reset_index()
merged = pd.merge(n_p1, n_p2, on = 'province')
result = merged[merged['count_p1'] > merged['count_p2']][['province','count_p1']].sort_values('count_p1', ascending = False)

print(results)



with a as(select province ,count(id) cnt1  from winemag_p1
group by 1
order by 2 desc),
  b as(select province ,count(id) cnt2  from winemag_p2
group by 1
order by 2 desc)
,  c as (select province,cnt1,cnt2  from a  
join b using(province))
select province,cnt1 from c 
where cnt1>cnt2




In [ ]:
# Find countries that are in winemag_p1 dataset but not in winemag_p2
# Find countries that are in winemag_p1 dataset but not in winemag_p2.
# Output distinct country names.
# Order records by the country in ascending order.

import pandas as pd
import numpy as np


p2 = winemag_p2[winemag_p2['country'].notnull()][['country']].drop_duplicates()
p1 = winemag_p1[winemag_p1['country'].notnull()][['country']].drop_duplicates()
df_all = p1.merge(p2.drop_duplicates(), on=['country'], 
                   how='left', indicator=True)
result = df_all[df_all['_merge'] == 'left_only'][['country']].sort_values('country')



select country from winemag_p1 
where country not in (select country from winemag_p2)





In [ ]:
Macedonian Vintages
Find the vintage years of all wines from the country of Macedonia. 
The year can be found in the 'title' column. Output the wine (i.e., the 'title') along with the year.
The year should be a numeric or int data type.


import pandas as pd
import numpy as np
macedonia = winemag_p2[winemag_p2['country'] == 'Macedonia']
macedonia['year'] = (macedonia['title'].str.extract('(\d{4,4})')).fillna(0).astype(int)
result = macedonia [['title','year']]


SELECT      title, split_part(title, ' ', 2) :: NUMERIC AS year
FROM winemag_p2
WHERE      country = 'Macedonia'

In [ ]:
# Median Price Of Wines
# Find the median price for each wine variety across both datasets. Output distinct varieties along with the corresponding median price.


# Import your libraries
import pandas as pd

p1 = winemag_p1[(winemag_p1['variety'].notnull()) & (winemag_p1['price'].notnull())][['variety','price']]
p2 = winemag_p2[(winemag_p2['variety'].notnull()) & (winemag_p2['price'].notnull())][['variety','price']]    
df = pd.concat([p1,p2])
df.groupby('variety')['price'].median().to_frame('median_price').reset_index()
 
    
    
    
with a as (
select  variety ,price  from winemag_p1 
union all 
select   variety,price    from winemag_p2)
select  variety,
PERCENTILE_CONT(0.5) within group (order by price) 
from a
group by 1    

In [ ]:
# Find the number of Bodegas outside of Spain by the country and region that produces wines with the blackberry taste
# Find the number of Bodegas outside of Spain by the country and region that produces wines with the blackberry taste.
# Output the country, region along with the number of bodegas.
# Order records by the number of bodegas in descending order.





import pandas as pd
import numpy as np

winemag_regions1 = winemag_p1[['country', 'region_1', 'winery', 'description']].rename({'region_1': 'region'}, axis='columns')
winemag_regions2 = winemag_p1[winemag_p1.region_1 != winemag_p1.region_2][['country', 'region_2', 'winery', 'description']].rename({'region_2': 'region'}, axis='columns')
winemag_all_regions = winemag_regions1.append(winemag_regions2, ignore_index=True)
bodegas = winemag_all_regions[(winemag_all_regions['country'] != 'Spain') & (winemag_all_regions['winery'].str.contains('bodega', case = False))
                & (winemag_all_regions['description'].str.contains('blackberry', case = False))]
result = bodegas.groupby(['country','region'])['winery'].nunique().to_frame('n_bodegas').reset_index().sort_values('n_bodegas', ascending = False)
result

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
winemag_p1.head()
df = winemag_p1
df1 = df[(df['description'].str.contains('blackberry', case = False)) & (df['winery'].str.contains('bodega', case = False))]
result = df1.groupby(['country','region_1'])['winery'].nunique().to_frame('n_bodegas').reset_index().sort_values('n_bodegas', ascending = False)



In [ ]:
# Import your libraries
import pandas as pd
import re
# Start writing code
winemag_p1[(winemag_p1.country != 'Spain') & 
           (winemag_p1.winery.str.contains(r'bodega',flags=re.I,na=False)) &
           (winemag_p1.description.str.contains(r'blackberry',flags=re.I))].groupby(['country','region_1']).winery.nunique().sort_values(ascending=False).reset_index()



# select country,region_1,count(distinct winery)
# from winemag_p1
# where country!='Spain' and winery ilike '%bodega%' and description ilike '%blackberry%'
# group by 1,2

In [ ]:
# Best Wines By Points-To-Price
# Find the wine with the highest points to price ratio. 
# Output the title, points, price, and the corresponding points-to-price ratio.




# Import your libraries
import pandas as pd
winemag_p1=pd.read_csv('winemag_p1.csv')
winemag_p2=pd.read_csv('winemag_p2.csv')

# Start writing code
df = winemag_p2[winemag_p2['price'].notnull()]
df['ratio']= df['points'] / df['price']
df[df['ratio']==df['ratio'].max()][['title', 'points', 'price', 'ratio']]




# select  title,points ,price,points/price points_price_ratio
# from winemag_p2
# where price is not null
# order by 4 desc
# limit 1 


In [6]:
# Wine Variety Revenues
# Find the total revenue made by each region from each variety of wine in that region.
# Output the region, variety, and total revenue.
# Take into calculation both region_1 and region_2 but be careful for cases where region_1 == region_2. 
# The revenue shouldn't be counted twice for cases like this.



import pandas as pd
import numpy as np
winemag_p1=pd.read_csv('winemag_p1.csv')
winemag_p2=pd.read_csv('winemag_p2.csv')
print(winemag_p1.info())
df1 = winemag_p1[['region_1', 'price', 'variety', 'id']].rename(columns={'region_1':'region'})
df2 = winemag_p1[['region_2', 'price', 'variety', 'id']].rename(columns={'region_2':'region'})
df = pd.concat([df1, df2], ignore_index=True).drop_duplicates()
df['price'] = df['price'].replace('', np.nan)
result1 = df.groupby(['region','variety'])['price'].sum()
print(result1)
result2=result1.to_frame('total').reset_index().sort_values('total', ascending=False)
print( '----')
print(result2)



# with a as ( select  region_1 region ,variety ,price
#   from winemag_p1
#   where region_1 is not null
#   UNION
#   select region_2 region ,variety,price  
#   from winemag_p1
#   where region_2 is not null)
#   select region,variety,sum(price)
#   from a
#   where price is not null
#   group by 1,2
#   order by 3 desc 
   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           100 non-null    int64  
 1   country      100 non-null    object 
 2   description  100 non-null    object 
 3   designation  64 non-null     object 
 4   points       100 non-null    int64  
 5   price        97 non-null     float64
 6   province     100 non-null    object 
 7   region_1     80 non-null     object 
 8   region_2     39 non-null     object 
 9   variety      100 non-null    object 
 10  winery       100 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 8.7+ KB
None
region                   variety                 
Alexander Valley         Cabernet Sauvignon           80.0
                         Merlot                       19.0
Anderson Valley          Pinot Noir                   52.0
Barbaresco               Nebbiolo                    242

In [7]:
# Find the number of wines with and without designations per country
# Find the number of wines with and without designations per country.
# Output the country along with the total without designations, total with designations, and the final total of both.




# Import your libraries
import pandas as pd

# Start writing code
winemag_p2['has_designation'] = winemag_p2['designation'].notna()

df = winemag_p2.groupby('country').has_designation.value_counts().unstack(fill_value=0)

df['total'] = df[False] + df[True]
df['total']
df



# with a as (select country,count(*) total_without_designation
# from winemag_p2
# where designation is null
# group by 1 
# order by 2 desc),
# b as (select country,count(*) total_with_designation
# from winemag_p2
# where designation is not null
# group by 1 
# order by 2 desc)
# select * ,total_with_designation+ total_without_designation  grand_total
# from a join b using(country)
# order by 2 


# select country,
# sum(case when designation is null then 1 else 0 end) as total_without_designation,
# sum(case when designation is not null then 1 else 0 end) as total_with_designation,
# count(*) as grand_total
# from winemag_p2
# group by country

has_designation,False,True,total
country,,,
Argentina,1,3,4
Australia,1,2,3
Austria,1,2,3
Brazil,0,1,1
Chile,1,1,2
France,5,16,21
Germany,0,3,3
Greece,0,1,1
Italy,4,6,10


In [8]:



# Top 3 Wineries In The World
# Find the top 3 wineries in each country based on the average points earned. Output the country along with the best, second best, and third best wineries. 
# If there is no second winery (NULL value) output 'No second winery' and if there is no third winery output 'No third winery'.

# Import your libraries
import pandas as pd

# Start writing code
first1 = winemag_p1.groupby(['winery','country'])['points'].mean().reset_index()

first1['rank_winery_by_points'] = first1.groupby('country')['points'].rank(method='first', ascending=False)

first1 = first1[first1['rank_winery_by_points']<=3]

result = first1.pivot_table(index=['country'],columns='rank_winery_by_points', values = 'winery', aggfunc='first').reset_index()

result

# with a as (select country,winery,  avg(points) avgpts
# from winemag_p1
#  group by 1,2
# order by 1,3 desc)
# ,b as(select country,concat(winery,' (',avgpts::int,')') winery  ,
# row_number() over(partition by country  order by avgpts desc) rn
# from a),
#  f as (select country,    winery   as top_winery from b  where rn=1),
#  s as (select country,    winery   as second_winery from b  where rn=2),
#  t as (select country,    winery   as third_winery from b  where rn=3)

# select distinct  b.country  ,  top_winery , 
# coalesce(second_winery,'No second winery') second_winery,  
# coalesce(third_winery,'No third winery')third_winery
# from  b left join f using (country)
#    left join s using (country)
# left join t using (country)
# order by 1

rank_winery_by_points,country,1.0,2.0,3.0
0,Argentina,Bodega Noem�_a de Patagonia,Bodega Norton,Rutini
1,Australia,Madison Ridge,NaN,NaN
2,Austria,Schloss Gobelsburg,H̦pler,NaN
3,Bulgaria,Targovishte,NaN,NaN
4,Chile,Alta��r,Fran̤ois Lurton,Santa Carolina
5,France,Domaine Faiveley,Roche de Bellene,Alain Jaume et Fils
6,Germany,Maximin Gr�_nh�_user,Bollig-Lehnert,A. Christmann
7,Greece,Santo Wines,NaN,NaN
8,Italy,Roagna,Il Mosnel,Ca' du Rabaj��
9,Moldova,Vinaria din Vale,NaN,NaN
